# Artificial Intelligence Nanodegree
## Recurrent Neural Network Projects

Welcome to the Recurrent Neural Network Project in the Artificial Intelligence Nanodegree! In this notebook, some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this project. You will not need to modify the included code beyond what is requested. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Implementation TODOs in this notebook

This notebook contains two problems, cut into a variety of TODOs.  Make sure to complete each section containing a TODO marker throughout the notebook.  For convenience we provide links to each of these sections below.

[TODO #1: Implement a function to window time series](#TODO_1)

[TODO #2: Create a simple RNN model using keras to perform regression](#TODO_2)

[TODO #3: Finish cleaning a large text corpus](#TODO_3)

[TODO #4: Implement a function to window a large text corpus](#TODO_4)

[TODO #5: Create a simple RNN model using keras to perform multiclass classification](#TODO_5)

[TODO #6: Generate text using a fully trained RNN model and a variety of input sequences](#TODO_6)


# Problem 1: Perform time series prediction 

In this project you will perform time series prediction using a Recurrent Neural Network regressor.  In particular you will re-create the figure shown in the notes - where the stock price of Apple was forecasted (or predicted) 7 days in advance.  In completing this exercise you will learn how to construct RNNs using Keras, which will also aid in completing the second project in this notebook.

The particular network architecture we will employ for our RNN is known as  [Long Term Short Memory (LSTM)](https://en.wikipedia.org/wiki/Long_short-term_memory), which helps significantly avoid technical problems with optimization of RNNs.  

## 1.1 Getting started

First we must load in our time series - a history of around 140 days of Apple's stock price.  Then we need to perform a number of pre-processing steps to prepare it for use with an RNN model.  First off, it is good practice to normalize time series - by normalizing its range.  This helps us avoid serious numerical issues associated how common activation functions (like tanh) transform very large (positive or negative) numbers, as well as helping us to avoid related issues when computing derivatives.

Here we normalize the series to lie in the range [0,1] [using this scikit function](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html), but it is also commonplace to normalize by a series standard deviation.

In [1]:
pip install tensorflow==2.16.2 tf-keras==2.16

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
import keras
import string


# Done: fill out the function below that transforms the input series 
# and window-size into a set of input/output pairs for use with our RNN model
''' ******************************************************** '''
def window_transform_series(series, window_size):
    # containers for input/output pairs
    X,y = [],[]
        
    for idx in range(len(series)-window_size):
        X.append(series[idx:idx+window_size])
    
    y = series[window_size:]
    
    # reshape each 
    X = np.asarray(X)
    X.shape = (np.shape(X)[0:2])
    y = np.asarray(y)
    y.shape = (len(y),1)
    
    
    return X,y


''' ******************************************************** '''
# Done: build an RNN to perform regression on our time series input/output data
def build_part1_RNN(window_size):
    RNN_model = Sequential()
    RNN_model.add(LSTM(5, input_shape=(window_size, 1)))
    RNN_model.add(Dense(1))
    return RNN_model
    #pass


''' ******************************************************** '''
### Done: return the text input with only ascii lowercase and the punctuation 
#   given below included.
def cleaned_text(text):
    punctuation = ['!', ',', '.', ':', ';', '?']
    extra1=['\xa0', '¢', '¨', '©', 'ã']
    extra2=['à', 'â', 'è', 'é']
    remove_set=(set(string.printable) | set(extra1) |set(extra2))-(set(string.ascii_lowercase)| set(punctuation) |set(' '))
    remove_set=list(remove_set)
    for c in remove_set:
        text=text.replace(c,' ')
    
    return text

''' ******************************************************** '''
### Done: fill out the function below that transforms the input text and 
##  window-size into a set of input/output pairs for use with our RNN model

def window_transform_text(text, window_size, step_size):
    # containers for input/output pairs
    inputs = []
    outputs = []
    idx = 0
    
    while idx<(len(text)-window_size):
        inputs.append(text[idx:idx+window_size])
        outputs.append(text[idx+window_size])
        idx += step_size
    
      
    # reshape each 
    '''
    inputs = np.asarray(inputs)
    inputs.shape = (len(inputs),1)
    outputs = np.asarray(outputs)
    outputs.shape = (len(outputs),1)
    '''
    
    return inputs,outputs

''' ******************************************************** '''
# TODO build the required RNN model: 
# a single LSTM hidden layer with softmax activation, categorical_crossentropy loss 
def build_part2_RNN(window_size, num_chars):
    model = Sequential()
    model.add(LSTM(200, input_shape=(window_size, num_chars)))
    model.add(Dense(num_chars))
    model.add(Activation('softmax'))
    return model

In [3]:
# Scaling function for the dataset
def scale_data(df, target_column):
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_scaled = scaler.fit_transform(df)
    return pd.DataFrame(df_scaled, columns=df.columns), scaler

# Inverse scaling for predictions
def inverse_scale(scaler, predictions, target_index):
    # Inverse scale only the target column
    full_scaled = np.zeros((len(predictions), scaler.n_features_in_))
    full_scaled[:, target_index] = predictions
    return scaler.inverse_transform(full_scaled)[:, target_index]

In [4]:
### Load in necessary libraries for data input and normalization
# Wi th inline backend, the output of plotting commands is displayed inline 
# within frontends directly below the code cell that produced it.
# The resulting plots will then also be stored in the notebook document.

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# %autoreload-reload all modules (except those excluded by %aimport) automatically now.
# %autoreload 2 - Reload all modules (except those excluded by %aimport) every time before executing the Python code typed.

%load_ext autoreload
%autoreload 2

#from my_answers import *

'''
%load_ext autoreload
%autoreload 2

from my_answers import *
'''

'\n%load_ext autoreload\n%autoreload 2\n\nfrom my_answers import *\n'

Lets take a quick look at the (normalized) time series we'll be performing predictions on.

In [5]:
import pandas as pd
airpass = pd.read_csv('AirPassengers.csv')
airpass['year'] = airpass['Month'].apply(lambda x: int(x[:4]))
airpass['month'] = airpass['Month'].apply(lambda x: int(x[5:]))
airpass.head()

,Month,Passengers,year,month
0,1949-01,112,1949,1
1,1949-02,118,1949,2
2,1949-03,132,1949,3
3,1949-04,129,1949,4
4,1949-05,121,1949,5


In [6]:
airpass.dtypes

Month         object
Passengers     int64
year           int64
month          int64
dtype: object

In [7]:
import tensorflow as tf
print(tf.__version__)

2.16.2


## 1.2  Cutting our time series into sequences

Remember, our time series is a sequence of numbers that we can represent in general mathematically as 

$$s_{0},s_{1},s_{2},...,s_{P}$$

where $s_{p}$ is the numerical value of the time series at time period $p$ and where $P$ is the total length of the series.  In order to apply our RNN we treat the time series prediction problem as a regression problem, and so need to use a sliding window to construct a set of associated input/output pairs to regress on.  This process is animated in the gif below.

<img src="timeseries_windowing_training.gif" width=600 height=600/>

For example - using a window of size T = 5 (as illustrated in the gif above) we produce a set of input/output pairs like the one shown in the table below

$$\begin{array}{c|c}
\text{Input} & \text{Output}\\
\hline \color{CornflowerBlue} {\langle s_{1},s_{2},s_{3},s_{4},s_{5}\rangle} & \color{Goldenrod}{ s_{6}} \\
\ \color{CornflowerBlue} {\langle s_{2},s_{3},s_{4},s_{5},s_{6} \rangle } & \color{Goldenrod} {s_{7} } \\
\color{CornflowerBlue}  {\vdots} & \color{Goldenrod} {\vdots}\\
\color{CornflowerBlue} { \langle s_{P-5},s_{P-4},s_{P-3},s_{P-2},s_{P-1} \rangle } & \color{Goldenrod} {s_{P}}
\end{array}$$

Notice here that each input is a sequence (or vector) of length 5 (and in general has length equal to the window size T) while each corresponding output is a scalar value.  Notice also how given a time series of length P and window size T = 5 as shown above, we created P - 5  input/output pairs.  More generally, for a window size T we create P - T such pairs.

Now its time for you to window the input time series as described above!  

<a id='TODO_1'></a>

**TODO:** Implement the function called **window_transform_series** in my_answers.py so that it runs a sliding window along the input series and creates associated input/output pairs.    Note that this function should input a) the series and b) the window length, and return the input/output subsequences.  Make sure to format returned input/output as generally shown in table above (where window_size = 5), and make sure your returned input is a numpy array.

-----

You can test your function on the list of odd numbers given below

In [8]:
odd_nums = np.array([1,3,5,7,9,11,13])

Here is a hard-coded solution for odd_nums.  You can compare its results with what you get from your **window_transform_series** implementation.

In [9]:
# run a window of size 2 over the odd number sequence and display the results
window_size = 2

X = []
X.append(odd_nums[0:2])
X.append(odd_nums[1:3])
X.append(odd_nums[2:4])
X.append(odd_nums[3:5])
X.append(odd_nums[4:6])

y = odd_nums[2:]

X = np.asarray(X)
y = np.asarray(y)
y = np.reshape(y, (len(y),1)) #optional

assert(type(X).__name__ == 'ndarray')
assert(type(y).__name__ == 'ndarray')
assert(X.shape == (5,2))
assert(y.shape in [(5,1), (5,)])

# print out input/output pairs --> here input = X, corresponding output = y
print ('--- the input X will look like ----')
print (X)

print ('--- the associated output y will look like ----')
print (y)

--- the input X will look like ----
[[ 1  3]
 [ 3  5]
 [ 5  7]
 [ 7  9]
 [ 9 11]]
--- the associated output y will look like ----
[[ 5]
 [ 7]
 [ 9]
 [11]
 [13]]


Again - you can check that your completed **window_transform_series** function works correctly by trying it on the odd_nums sequence - you should get the above output.

In [10]:
### Done: implement the function window_transform_series in the file my_answers.py
#from my_answers import window_transform_series
window_size = 2
X,y = window_transform_series(series = odd_nums,window_size = window_size)

# print out input/output pairs --> here input = X, corresponding output = y
print ('--- the input X from my implementation looks like ----')
print (X)

print ('--- the associated output y from my implementation looks like ----')
print (y)

--- the input X from my implementation looks like ----
[[ 1  3]
 [ 3  5]
 [ 5  7]
 [ 7  9]
 [ 9 11]]
--- the associated output y from my implementation looks like ----
[[ 5]
 [ 7]
 [ 9]
 [11]
 [13]]


With this function in place apply it to the series in the Python cell below.  We use a window_size = 7 for these experiments.

In [11]:
airpass = airpass[['Passengers','year','month']]
airpass.shape

(144, 3)

In [12]:
# Scaling the data
from sklearn.preprocessing import MinMaxScaler
target_column = 'Passengers'
df_scaled, scaler = scale_data(airpass, target_column)

In [13]:
import numpy as np
import pandas as pd

def prepare_rnn_data(df, input_window, output_window, target_columns=None):
    """
    Prepares multivariate time series data for RNN processing using a rolling window approach.
    
    Parameters:
        df (pd.DataFrame): The multivariate time series as a pandas DataFrame.
        input_window (int): Number of time steps used as input (X).
        output_window (int): Number of time steps used as output (y).
        target_columns (list): Optional list of target columns. If None, uses all columns.
        
    Returns:
        X (np.array): Input sequences of shape (num_samples, input_window, num_features).
        y (np.array): Output labels of shape (num_samples, output_window, num_features).
    """
    
    if target_columns is None:
        target_columns = df.columns  # Use all columns if target_columns is not provided
    
    X, y = [], []
    
    for i in range(len(df) - input_window - output_window + 1):
        # Get the input window (X) and the corresponding output window (y)
        X.append(df.iloc[i:i+input_window].values)
        y.append(df.iloc[i+input_window:i+input_window+output_window][target_columns].values)
    
    return np.array(X), np.array(y)



input_window = 10
output_window = 1
X, y = prepare_rnn_data(df_scaled, input_window, output_window, target_columns=['Passengers'])

print("Input shape (X):", X.shape)
print("Output shape (y):", y.shape)

Input shape (X): (134, 10, 3)
Output shape (y): (134, 1, 1)


## 1.3  Splitting into training and testing sets

In order to perform proper testing on our dataset we will lop off the last 1/3 of it for validation (or testing).  This is that once we train our model we have something to test it on (like any regression problem!).  This splitting into training/testing sets is done in the cell below.

Note how here we are **not** splitting the dataset *randomly* as one typically would do when validating a regression model.  This is because our input/output pairs *are related temporally*.   We don't want to validate our model by training on a random subset of the series and then testing on another random subset, as this simulates the scenario that we receive new points *within the timeframe of our training set*.  

We want to train on one solid chunk of the series (in our case, the first full 2/3 of it), and validate on a later chunk (the last 1/3) as this simulates how we would predict *future* values of a time series.

In [14]:
# split our dataset into training / testing sets
train_test_split = int(np.ceil(2*len(y)/float(3)))   # set the split point

# partition the training set
X_train = X[:train_test_split,:]
y_train = y[:train_test_split]

# keep the last chunk for testing
X_test = X[train_test_split:,:]
y_test = y[train_test_split:]
train_test_split

90

<a id='TODO_2'></a>

## 1.4  Build and run an RNN regression model

Having created input/output pairs out of our time series and cut this into training/testing sets, we can now begin setting up our RNN.  We use Keras to quickly build a two hidden layer RNN of the following specifications

- layer 1 uses an LSTM module with 5 hidden units (note here the input_shape = (window_size,1))
- layer 2 uses a fully connected module with one unit
- the 'mean_squared_error' loss should be used (remember: we are performing regression here)

This can be constructed using just a few lines - see e.g., the [general Keras documentation](https://keras.io/getting-started/sequential-model-guide/) and the [LSTM documentation in particular](https://keras.io/layers/recurrent/) for examples of how to quickly use Keras to build neural network models.  Make sure you are initializing your optimizer given the [keras-recommended approach for RNNs](https://keras.io/optimizers/) 

(given in the cell below).  (remember to copy your completed function into the script *my_answers.py* function titled *build_part1_RNN* before submitting your project)

In [15]:
np.shape(X)[1:]

(10, 3)

In [16]:
X

array([[[0.01544402, 0.        , 0.        ],
        [0.02702703, 0.        , 0.09090909],
        [0.05405405, 0.        , 0.18181818],
        ...,
        [0.08494208, 0.        , 0.63636364],
        [0.06177606, 0.        , 0.72727273],
        [0.02895753, 0.        , 0.81818182]],

       [[0.02702703, 0.        , 0.09090909],
        [0.05405405, 0.        , 0.18181818],
        [0.04826255, 0.        , 0.27272727],
        ...,
        [0.06177606, 0.        , 0.72727273],
        [0.02895753, 0.        , 0.81818182],
        [0.        , 0.        , 0.90909091]],

       [[0.05405405, 0.        , 0.18181818],
        [0.04826255, 0.        , 0.27272727],
        [0.03281853, 0.        , 0.36363636],
        ...,
        [0.02895753, 0.        , 0.81818182],
        [0.        , 0.        , 0.90909091],
        [0.02702703, 0.        , 1.        ]],

       ...,

       [[0.58108108, 0.90909091, 1.        ],
        [0.6042471 , 1.        , 0.        ],
        [0.55405405, 1

In [17]:
from tensorflow.keras import Input
def build_part3_RNN(window_size):
    RNN_model = Sequential()
    RNN_model.add(Input(shape=np.shape(X)[1:]))
    RNN_model.add(LSTM(50,return_sequences=True))
    RNN_model.add(LSTM(10))
    RNN_model.add(Dense(1))
    return RNN_model
    #pass

In [18]:
### Done: create required RNN model
# import keras network libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow.keras

# given - fix random seed - so we can all reproduce the same results on our default time series
np.random.seed(0)

window_size = 10
# TODO: implement build_part1_RNN in my_answers.py
#from my_answers import build_part1_RNN
model = build_part3_RNN(window_size)

# build model using keras documentation recommended optimizer initialization
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# compile the model
model.compile(loss='mean_squared_error', optimizer=optimizer)

/Users/mattsalomon/miniconda3/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


With your model built you can now fit the model by activating the cell below!  Note: the number of epochs (np_epochs) and batch_size are preset (so we can all produce the same results).  You can choose to toggle the verbose parameter - which gives you regular updates on the progress of the algorithm - on and off by setting it to 1 or 0 respectively.

In [19]:
# run your model!
model.fit(X_train, y_train, epochs=500, batch_size=25, verbose=0)

## 1.5  Checking model performance

With your model fit we can now make predictions on both our training and testing sets.

In [20]:
df_scaled.shape

(144, 3)

In [22]:
import numpy as np
import tensorflow as tf
def prepare_forecat_data(df, input_window, output_window, target):
    """
    Prepares multivariate time series data for RNN processing using a rolling window approach.
    
    Parameters:
        df (pd.DataFrame): The multivariate time series as a pandas DataFrame.
        input_window (int): Number of time steps used as input (X).
        output_window (int): Number of time steps used as output (y).
        
    Returns:
        X (np.array): Input sequences of shape (num_samples, input_window, num_features).
    """
    
#     if target_columns is None:
#         target_columns = df.columns  # Use all columns if target_columns is not provided
    
    exogenous, data = [], []
    
    for i in range(len(df) - input_window - output_window + 1):
        exogenous.append(df[[x for x in df.columns if x!=target]].iloc[i:i+input_window].values)
        #data.append(df[[x for x in df.columns if x==target ]].iloc[i:i+input_window].values)
        data.append(df.iloc[i:i+input_window][target].values)
    return np.array(data), np.array(exogenous)

def rolling_window_prediction(model, data, exogenous_inputs,df_scaled, window_size, forecast_steps=1):
    """
    Generates rolling window time series predictions using a pre-trained LSTM model.
    
    Parameters:
        model: Pre-trained TensorFlow model (LSTM).
        data: numpy array of the main time series data (shape: [n_samples, n_features]).
        exogenous_inputs: numpy array of exogenous inputs (shape: [n_samples, n_exogenous_features]).
        window_size: Size of the rolling window.
        forecast_steps: Number of time steps to predict at each iteration.
    
    Returns:
        predictions: List of predicted values.
    """
    
    # Initialize a list to store predictions
    predictions = []
    data = np.expand_dims(data, axis=-1) 
    # Combine time series data and exogenous inputs
    #data_internal=data
    combined_data = np.concatenate((data, exogenous_inputs),axis=2)
    df_scaled_interal=df_scaled
    # Number of time steps we can predict using rolling windows
    num_samples = len(data) - window_size
    print(combined_data.shape)
    for i in range(num_samples):
        # Extract the input window (input for the model)
        #input_window = combined_data[i:i+window_size]
        
        # Reshape to match model input shape (batch_size, window_size, n_features)
        #input_window = np.expand_dims(input_window, axis=0)  # Add batch dimension
        #print(input_window.shape)
        # Get the prediction (the output is a sequence, take the last step if forecast_steps > 1)
        #prediction = model.predict(input_window)
        prediction = model.predict(combined_data[i,:,:].reshape(1,10,3))
        
        # Append prediction
        predictions.append(prediction[0][-1] if forecast_steps > 1 else prediction[0])
        print("what you need is",df_scaled_interal.iloc[i+1+window_size])
        print("df_scaled_interal shape pre",df_scaled_interal.shape)
        if i>=train_test_split:
            df_scaled_interal.iloc[i+1+window_size] = prediction        
            print("df_scaled_interal shape post",df_scaled_interal.shape)
        data_internal,_  = prepare_forecat_data(df_scaled_interal, input_window=10, output_window=1, target=['Passengers']) 
        print(data_internal.shape, exogenous_inputs.shape)

        combined_data = np.concatenate((data_internal, exogenous_inputs),axis=2)  
        print(combined_data.shape)
        
    return np.array(predictions)

exogenous_inputs = X[:,:,[1,2]]
data = X[:,:,0]
#Define window size and forecast steps
window_size = 10
forecast_steps = 1

predictions = rolling_window_prediction(model, data, exogenous_inputs,df_scaled, window_size, forecast_steps)

(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
what you need is Passengers    0.027027
year          0.000000
month         1.000000
Name: 11, dtype: float64
df_scaled_interal shape pre (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
what you need is Passengers    0.021236
year          0.090909
month         0.000000
Name: 12, dtype: float64
df_scaled_interal shape pre (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
what you need is Passengers    0.042471
year          0.090909
month         0.090909
Name: 13, dtype: float64
df_scaled_interal shape pre (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
what you need is Passengers    0.071429
year          0.090909
month         0.181818
Name: 14, dtype: float64
df_scaled_interal shape pre (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
what you need is Passengers    0.059846
year     

(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
what you need is Passengers    0.266409
year          0.272727
month         0.636364
Name: 43, dtype: float64
df_scaled_interal shape pre (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
what you need is Passengers    0.202703
year          0.272727
month         0.727273
Name: 44, dtype: float64
df_scaled_interal shape pre (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
what you need is Passengers    0.167954
year          0.272727
month         0.818182
Name: 45, dtype: float64
df_scaled_interal shape pre (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
what you need is Passengers    0.131274
year          0.272727
month         0.909091
Name: 46, dtype: float64
df_scaled_interal shape pre (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
what you need is Passen

(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
what you need is Passengers    0.318533
year          0.545455
month         0.272727
Name: 75, dtype: float64
df_scaled_interal shape pre (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
what you need is Passengers    0.320463
year          0.545455
month         0.363636
Name: 76, dtype: float64
df_scaled_interal shape pre (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
what you need is Passengers    0.407336
year          0.545455
month         0.454545
Name: 77, dtype: float64
df_scaled_interal shape pre (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
what you need is Passengers    0.501931
year          0.545455
month         0.545455
Name: 78, dtype: float64
df_scaled_interal shape pre (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
what you need is Passen

(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
what you need is Passengers    0.454886
year          0.454886
month         0.454886
Name: 106, dtype: float64
df_scaled_interal shape pre (144, 3)
df_scaled_interal shape post (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
what you need is Passengers    0.398402
year          0.398402
month         0.398402
Name: 107, dtype: float64
df_scaled_interal shape pre (144, 3)
df_scaled_interal shape post (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
what you need is Passengers    0.457118
year          0.457118
month         0.457118
Name: 108, dtype: float64
df_scaled_interal shape pre (144, 3)
df_scaled_interal shape post (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
what you need is Passengers    0.489636
year          0.489636
month         0.489636
Name: 109, dtype: float64
df_scaled_interal s

(134, 10, 1) (134, 10, 2)
(134, 10, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
what you need is Passengers    0.666351
year          0.666351
month         0.666351
Name: 134, dtype: float64
df_scaled_interal shape pre (144, 3)
df_scaled_interal shape post (144, 3)
(134, 10, 1) (134, 10, 2)
(134, 10, 3)


In [23]:
# print out training and testing errors
training_error = model.evaluate(X_train, y_train, verbose=0)
print('training error = ' + str(training_error))

testing_error = model.evaluate(X_test, y_test, verbose=0)
print('testing error = ' + str(testing_error))

training error = 0.00046222706441767514
testing error = 0.002888695104047656


In [ ]:
# Inverse scaling for predictions
def inverse_scale(scaler, predictions, target_index):
    # Inverse scale only the target column
    full_scaled = np.zeros((len(predictions), scaler.n_features_in_))
    full_scaled[:, target_index] = predictions.flatten()
    return scaler.inverse_transform(full_scaled)[:, target_index]

In [ ]:
target_column = 'Passengers'
target_index = list(airpass.columns).index(target_column)

In [ ]:
predictions.shape,airpass.shape

In [ ]:
train_predict = inverse_scale(scaler, predictions, 0)[0:train_test_split]
test_predict = inverse_scale(scaler, predictions, 0)[train_test_split:]

In [ ]:
### Plot everything - the original series as well as predictions on training and testing sets
import matplotlib.pyplot as plt
%matplotlib inline

# plot original series
plt.plot(airpass['Passengers'],color = 'g')

# plot training set prediction
split_pt = train_test_split + window_size 
plt.plot(np.arange(window_size,split_pt,1),train_predict[:],color = 'b')

# plot testing set prediction
plt.plot(np.arange(split_pt,split_pt + len(test_predict),1),test_predict,color = 'r')

# pretty up graph
plt.xlabel('day')
plt.ylabel('(normalized) price of Apple stock')
plt.legend(['original series','training fit','testing fit'],loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

**Note:** you can try out any time series for this exercise!  If you would like to try another see e.g., [this site containing thousands of time series](https://datamarket.com/data/list/?q=provider%3Atsdl) and pick another one!

In [24]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(airpass['Passengers'][100:134], test_predict) 
print(f'test mape is: { round(mape*100,1)} %')

NameError: name 'test_predict' is not defined